# Labelling & Cleaning


In [1]:
import pandas as pd

# Load raw Strava data
df = pd.read_csv("../data/raw/strava_activities_raw_latest.csv")

print("Total activities:", len(df))
df.head()


Total activities: 190


,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,...,Intensity,Average Grade Adjusted Pace,Timer Time,Total Cycles,Recovery,With Pet,Competition,Long Run,For a Cause,Media
0,12403983821,"Sep 14, 2024, 1:29:18 AM",Morning Run,Run,NaN,5989,6.80,NaN,NaN,False,...,NaN,1.401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13647748281,"Feb 16, 2025, 11:51:16 PM",JALAAANN,Walk,NaN,4915,5.62,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13887014028,"Mar 15, 2025, 9:50:11 AM",jalan lagih,Walk,NaN,1816,2.11,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13896731706,"Mar 16, 2025, 9:49:27 AM",JALAANN LAGEH,Walk,NaN,4666,6.05,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13934062015,"Mar 20, 2025, 8:49:32 AM",first run,Run,NaN,3204,6.18,NaN,NaN,False,...,NaN,2.059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.columns

Index(['Activity ID', 'Activity Date', 'Activity Name', 'Activity Type',
       'Activity Description', 'Elapsed Time', 'Distance', 'Max Heart Rate',
       'Relative Effort', 'Commute', 'Activity Private Note', 'Activity Gear',
       'Filename', 'Athlete Weight', 'Bike Weight', 'Elapsed Time.1',
       'Moving Time', 'Distance.1', 'Max Speed', 'Average Speed',
       'Elevation Gain', 'Elevation Loss', 'Elevation Low', 'Elevation High',
       'Max Grade', 'Average Grade', 'Average Positive Grade',
       'Average Negative Grade', 'Max Cadence', 'Average Cadence',
       'Max Heart Rate.1', 'Average Heart Rate', 'Max Watts', 'Average Watts',
       'Calories', 'Max Temperature', 'Average Temperature',
       'Relative Effort.1', 'Total Work', 'Number of Runs', 'Uphill Time',
       'Downhill Time', 'Other Time', 'Perceived Exertion', 'Type',
       'Start Time', 'Weighted Average Power', 'Power Count',
       'Prefer Perceived Exertion', 'Perceived Relative Effort', 'Commute.1',
    

In [3]:
# Keep running activities only
runs = df[df["Activity Type"] == "Run"].copy()

print("Total runs:", len(runs))
runs[["Activity Date", "Distance", "Moving Time", "Average Heart Rate"]].head()


Total runs: 101


,Activity Date,Distance,Moving Time,Average Heart Rate
0,"Sep 14, 2024, 1:29:18 AM",6.80,4909.0,NaN
4,"Mar 20, 2025, 8:49:32 AM",6.18,3047.0,NaN
6,"Mar 25, 2025, 9:06:11 AM",5.16,2407.0,NaN
7,"Mar 27, 2025, 9:14:50 AM",2.87,1178.0,NaN
8,"Mar 29, 2025, 9:11:40 AM",3.35,1391.0,NaN


In [4]:
# drop run without HR data 
runs = runs.dropna(subset=["Average Heart Rate"])

# convert date (Strava-safe)
runs["Activity Date"] = pd.to_datetime(
    runs["Activity Date"],
    format="mixed"
)

# sort by date
runs = runs.sort_values(by="Activity Date")

print("Runs with HR data:", len(runs))
runs.head()
runs.to_csv("../data/processed/runs_with_hr.csv", index=False)


Runs with HR data: 78


## Pace calculation

In [5]:
# avg speed is in m/s -> convert to pace in min/km
runs["pace_min_per_km"] = 1000 / (runs["Average Speed"] * 60)
runs[["Average Speed", "pace_min_per_km"]].head()

,Average Speed,pace_min_per_km
27,2.409,6.918500
28,2.364,7.050197
29,2.176,7.659314
30,2.176,7.659314
31,1.819,9.162544


## what will happend 

1. no analysis
2. only labelling
    - 2 type of labelling = 
        - training phase (body condition & program)
        - intensity zone (MAF - Based)
    

## Traning Phase Labelling

in this part there will be 4 label 

1. pre_injury  (before injury) old baseline before ITBS
2. post_injury  (September 2025 3 times a week run) 
3. return_to_run (October 2025 3 times a week run till the end of the 2025)
4. building_intensity (MAF block progressve)

prety much like that.. hopefully i know what i'm doing

In [6]:
runs["training_phase"] = "building_intensity"

In [7]:
# pre-injury
runs.loc[
    runs["Activity Date"] < "2025-09-01",
    "training_phase"
] = "pre_injury"

# post injury
runs.loc[
    (runs["Activity Date"] >= "2025-09-01") &
    (runs["Activity Date"] < "2025-10-01"),
    "training_phase"
] = "post_injury"

# return_to_run
runs.loc[
    (runs["Activity Date"] >= "2025-10-01") &
    (runs["Activity Date"] < "2025-11-01"),
    "training_phase"
] = "return_to_run"



## Intensity Zone (MAF BASED) Labeling

zone definition = 

1. maf_pure ->           avg hr <= 154 
2. maf_tolerated ->      154 < avg hr <= 164
3. non_maf ->            avg hr > 164   
4. no_hr ->              HR data empty (NA)




In [8]:
MAF_HR = 154
MAF_TOLERANCE = 10


runs["maf_zone"] = "no_hr"

In [9]:
runs["maf_zone"] = "non_maf"

runs.loc[
    runs["Average Heart Rate"] <= MAF_HR,
    'maf_zone'
] = "maf_pure"

runs.loc[
    (runs["Average Heart Rate"] > MAF_HR) &
    (runs["Average Heart Rate"] <= MAF_HR + MAF_TOLERANCE),
    "maf_zone"
] = "maf_tolerated"

In [10]:
runs.groupby(["training_phase", "maf_zone"]).size()



training_phase      maf_zone     
building_intensity  maf_pure         14
                    maf_tolerated    12
post_injury         maf_tolerated     1
                    non_maf           1
pre_injury          maf_pure          7
                    maf_tolerated    18
                    non_maf          16
return_to_run       maf_pure          9
dtype: int64

## Training Complience Check

In [11]:
building = runs.loc[
    runs["training_phase"] == "building_intensity"
].copy()

building["maf_zone"].value_counts(normalize=True) * 100


maf_zone
maf_pure         53.846154
maf_tolerated    46.153846
Name: proportion, dtype: float64

In [12]:
tolerated_ratio = (
    building["maf_zone"]
    .value_counts(normalize=True)
    .get("maf_tolerated", 0)
)

if tolerated_ratio > 0.4:
    print("🚨 WARNING: MAF tolerated > 40% — hold volume & slow down")
elif tolerated_ratio > 0.3:
    print("⚠️ CAUTION: MAF tolerated > 30% — monitor closely")
else:
    print("✅ MAF compliance OK")


🚨 WARNING: MAF tolerated > 40% — hold volume & slow down


well, according to that i kinda in the yellow zone, i need to keep it easy for a while, and monitor it closely. to anyone dont get injured.

In [13]:
runs.to_csv("../data/processed/runs_labeled.csv", index=False)

so after the first performance analysis i can see that i have an ideal avg heart rate to use insteadof the maf heartrate or the maf zones

and cause of that im gonna make a new labeling for the heartrate zone for maf. it will be like the code below that im about to type.

In [14]:
runs["maf_band"] = "no_hr"

runs.loc[
    runs["Average Heart Rate"] <= 155,
    "maf_band"
] = "maf_clean"

runs.loc[
    (runs["Average Heart Rate"] > 155) &
    (runs["Average Heart Rate"] <= 160),
    "maf_band"
] = "maf_tolerated"

runs.loc[
    runs["Average Heart Rate"] > 160,
    "maf_band"
] = "non_maf"


its basiclly almost the same or just the same but the bpm is just different. its because according to the performance analysis i have an ideal bpm which is 155-160 bpm because there is so many runs in that area of bpm that just make me able to run faster and improve my aerobic capacity. for the simple thing is at the avg heartrate of <= 155 or 155 to 160 bpm i have a good running performance and actually improve my aerobic capacity to a higher level. hopefully you understand what i mean

In [15]:
runs.groupby(["training_phase", "maf_zone"]).size()

training_phase      maf_zone     
building_intensity  maf_pure         14
                    maf_tolerated    12
post_injury         maf_tolerated     1
                    non_maf           1
pre_injury          maf_pure          7
                    maf_tolerated    18
                    non_maf          16
return_to_run       maf_pure          9
dtype: int64

In [16]:
runs.to_csv("../data/processed/runs_labeledv2.csv", index=False)